<a href="https://colab.research.google.com/github/imthelizardking/CMP784--Final--Project/blob/main/suggestion_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import and install libraries, dependencies

In [1]:
import tensorflow as tf
import numpy as np
import os
import json
import collections
import pickle
from tqdm import tqdm
import pylab
use_colab = True # belki çıkacak
TINY = 1e-30
EPS = 1e-4
nax = np.newaxis

Connect to gdrive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Import data

In [3]:
# import data, from gdrive
dirJson = "/content/gdrive/MyDrive/cmp784_project_literature/colab repo/test_inputs/"
%cd "/content/gdrive/MyDrive/cmp784_project_literature/colab repo/test_inputs"

/content/gdrive/MyDrive/cmp784_project_literature/colab repo/test_inputs


In [13]:
json_files = [pos_json for pos_json in os.listdir(dirJson) if pos_json.endswith('.json')] # get all .json anno. files from dir
NUM_FEATURES = 13 # number of features (types, color, style etc.)
labels_vector = np.zeros((len(json_files),NUM_FEATURES)) # one hot encode matrix
labels_vector_ids = np.zeros((len(json_files),NUM_FEATURES))
labels_vector_ids_size = np.zeros(len(json_files))
counter_outer = 0 # anno. counter
# dictionary of clothing types, will be expanded
#typeDict ={
#    "vest": 0, #sets index of type to 0, if item exist index0 will be 1
#    "sling dress": 1,
#    "short sleeve dress": 2
#}
typeDict = {
    "short sleeve top": 0, # shifted -1 due to 
    "long sleeve top": 1,
    "short sleeve outwear": 2,
    "long sleeve outwear": 3,
    "vest": 4,
    "sling": 5,
    "shorts": 6,
    "trousers": 7,
    "skirt": 8,
    "short sleeve dress": 9,
    "long sleeve dres": 10,
    "vest dress": 11,
    "sling dress": 12,
}
for json_ in json_files:
  jsonFile = open(json_, 'r')
  values = json.load(jsonFile)
  jsonFile.close()
  counter_inner = 0
  for (k,v) in values.items():
    if k.startswith('item'):    
      labels_vector[counter_outer][typeDict[v["category_name"]]] = 1
      labels_vector_ids[counter_outer][counter_inner] = typeDict[v["category_name"]]
      counter_inner = counter_inner + 1
  labels_vector_ids_size[counter_outer] = counter_inner
  counter_outer = counter_outer + 1
#print(labels_vector)
print(labels_vector_ids_size)
labels_vector_ids = labels_vector_ids + 1 # hepsine +1 yapmak gerekebilir, 1 olsun min
print(labels_vector_ids)

[1. 2. 1. 1. 1. 1. 1. 2. 1. 2. 1. 2. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1.
 1. 1. 1. 1. 2. 1. 2.]
[[ 9.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [12.  4.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 8.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 9.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 9.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 8.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 6.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 9.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 9.  7.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [12.  4.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  9.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [11.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0

Create co-occurence matrix:

In [16]:
dataset_size = (len(json_files))
def calculate_log_co_occurence(labels_vector_ids, labels_vector_ids_size, ratings_heart, ratings_comment):
  "Compute the log-co-occurence matrix for our data."
  log_co_occurence = np.zeros((dataset_size, dataset_size, dataset_size)) # hepsi dataset_size ya da labels_vector_ids size olsun!!!

  for label_vector in range(len(labels_vector_ids)):
    i,j,k = 0,0,0
    i,j,k = label_vector[0], label_vector[1], label_vector[2]
    log_co_occurence[i][j][k] +=1
    # add ratings assignment with scaler/normalizer

  log_co_occurence = np.multiply(log_co_occurence, rating_matrix)
  delta_smoothing = 0.5  # A hyperparameter.  You can play with this if you want.
  log_co_occurence += delta_smoothing  # Add delta so log doesn't break on 0's.
  # maybe normalize
  log_compatibility_matrix = np.log(log_co_occurence)
  return log_compatibility_matrix

In [ ]:
[data_train, data_valid] =  np.array_split(labels_vector_ids,2)
log_compatibility_matrix_train = calculate_log_co_occurence(data_train)
log_co_occurence_valid = calculate_log_co_occurence(data_valid)

Train helpers:

In [ ]:
def loss_GLoVE(W, b, log_co_occurence):
  "Compute the GLoVE loss."
  n,_ = log_co_occurence.shape
  return np.sum((W @ W.T + b @ np.ones([1,n]) + np.ones([n,1])@b.T - log_co_occurence)**2)

def grad_GLoVE(W,  b, log_co_occurence):
  "Return the gradient of GLoVE objective w.r.t W and b."
  "INPUT: W - Vxd; b - Vx1; log_co_occurence: VxV"
  "OUTPUT: grad_W - Vxd; grad_b - Vx1"
  n,_ = log_co_occurence.shape
  ###########################   YOUR CODE HERE  ##############################
  grad_W = 2 * (W.T @ (W @ W.T + b @ np.ones([1,n]) + np.ones([n,1])@b.T - log_co_occurence).T).T
  grad_b = 2 * (np.ones([1,250]) @ (W @ W.T + b @ np.ones([1,n]) + np.ones([n,1])@b.T - log_co_occurence).T).T
  ############################################################################
  return grad_W, grad_b

Trainer:

In [18]:
def train(W, b, log_co_occurence_train, log_co_occurence_valid, n_epochs, do_print=False):
  "Traing W and b according to GLoVE objective."
  n,_ = log_co_occurence_train.shape
  learning_rate = 0.5 / n  # 0.5 A hyperparameter.  You can play with this if you want.
  for epoch in range(n_epochs):
    grad_W, grad_b = grad_GLoVE(W, b, log_co_occurence_train)
    W -= learning_rate * grad_W
    b -= learning_rate * grad_b
    train_loss, valid_loss = loss_GLoVE(W, b, log_co_occurence_train), loss_GLoVE(W, b, log_co_occurence_valid)
    if do_print:
      print(f"Train Loss: {train_loss}, valid loss: {valid_loss}, grad_norm: {np.sum(grad_W**2)}")
  return W, b, train_loss, valid_loss

Set Trainer Parameters and Train

In [ ]:
np.random.seed(1)
n_epochs = 500  # 500 A hyperparameter.  You can play with this if you want.
embedding_dims = np.array([2, 3, 4, 5, 6, 7, 8, 10, 12, 14, 16, 256])  # Play with this
final_train_losses, final_val_losses = [], []  # Store the final losses for graphing
W_final_2d, b_final_2d = None, None
do_print = True  # If you want to see diagnostic information during training
for embedding_dim in tqdm(embedding_dims):
  init_variance = 0.5  # A hyperparameter.  You can play with this if you want.
  W = init_variance * np.random.normal(size=(250, embedding_dim))
  b = init_variance * np.random.normal(size=(250, 1))
  if do_print:
    print(f"Training for embedding dimension: {embedding_dim}")
  W_final, b_final, train_loss, valid_loss = train_GLoVE(W, b, log_co_occurence_train, log_co_occurence_valid, n_epochs, do_print=False)
  if embedding_dim == 2:
    # Save a parameter copy if we are training 2d embedding for visualization later
    W_final_2d = W_final
    b_final_2d = b_final
  final_train_losses += [train_loss]
  final_val_losses += [valid_loss]
  if do_print:
    print(f"Final validation loss: {valid_loss}")

Plot training metrics:

In [ ]:
pylab.loglog(embedding_dims, final_train_losses)
pylab.xlabel("Embedding Dimension")
pylab.ylabel("Training Loss")
pylab.legend()

In [ ]:
pylab.loglog(embedding_dims, final_val_losses)
pylab.xlabel("Embedding Dimension")
pylab.ylabel("Validation Loss")
pylab.legend()

**PREDICTOR**

Process input

In [ ]:
test_input = np.array([1,0,0,0,0,0,0,0,0,0,0,0,0]) + 1
test_input_processed = np.zeros(len(test_input))
counter_distinct = 0
for e in range(len(test_input)):
  if e!=0:
    test_input_processed[counter_distinct] = e
    counter_distinct = counter_distinct + 1

Create Combinations of Input

In [ ]:
def createCombinations(test_input_processed, counter_distinct):
  counter_array = 0
  if counter_distinct==0 or counter_distinct==1:
    print("Can't make any prediction for item size smaller than 2")
    return
  else:
    if counter_distinct==2:
      test_array = np.zeros(2)
      test_array[0] = test_array[0] + test_input_processed[0]
      test_array[1] = test_array[1] + test_input_processed[1]
    elif counter_distinct==3:
      test_array = np.zeros(3)
      test_array[0] = test_array[0] + test_input_processed[0]
      test_array[1] = test_array[1] + test_input_processed[1]
      test_array[2] = test_array[2] + test_input_processed[2]              

Predictor:

In [ ]:
def predictor():
  

Run Preditor for each combination

Choose b/w most probable results

Feed suggestion into language model

Output image, bb and textual